<a href="https://colab.research.google.com/github/ariflaksito/recsys/blob/master/05_cb_knnanime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Content-based RS
CB RS using KNN for data Anime movies<br />
Code: https://github.com/sathvikb007/Anime_Recommendation/blob/master/AnimeRecommender.ipynb

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
anime = pd.read_csv('/content/drive/My Drive/Colab Data/anime.csv')

In [5]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [6]:
anime['type'].unique()

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA', nan], dtype=object)

In [7]:
anime[(anime.episodes=="Unknown") & (anime.type=="OVA")]

,anime_id,name,genre,type,episodes,rating,members
8200,33566,Baki,"Action, Martial Arts, Shounen",OVA,Unknown,8.27,1448
8521,24985,Elite Jack!!,"School, Shoujo",OVA,Unknown,5.29,141
8977,24501,Inui-san!,"Comedy, Shoujo",OVA,Unknown,5.20,106
9689,15211,Nazotoki-hime wa Meitantei♥,"Mystery, Shoujo",OVA,Unknown,5.62,194
10942,33564,Armed Blue: Gunvolt,"Action, Sci-Fi, Super Power",OVA,Unknown,NaN,2549
11008,33797,ID-0,Mecha,OVA,Unknown,NaN,292
11022,33191,Kishibe Rohan wa Ugokanai,"Action, Adventure, Mystery, Supernatural",OVA,Unknown,NaN,2607
11031,34151,Landreaall,"Action, Adventure, Fantasy, Martial Arts, Romance",OVA,Unknown,NaN,414
11093,21305,Trick or Alice,Fantasy,OVA,Unknown,NaN,4481
11100,33064,Uchuu Senkan Yamato 2202: Ai no Senshi-tachi,"Action, Drama, Military, Sci-Fi, Space",OVA,Unknown,NaN,2058


In [8]:
anime[(anime.episodes=="Unknown") & (anime.type=="Movie")]

,anime_id,name,genre,type,episodes,rating,members
8142,30617,Animegatari,Comedy,Movie,Unknown,5.36,1308
10940,33820,Ao Oni The Animation (Movie),Horror,Movie,Unknown,NaN,1416
10947,32086,Blame! Movie,"Action, Drama, Mecha, Psychological, Sci-Fi, S...",Movie,Unknown,NaN,3251
10996,33783,Godzilla,"Action, Adventure, Fantasy, Sci-Fi",Movie,Unknown,NaN,1022


## Pre-processing data 

In [0]:
# Set nilai episodes = 1 dimana type movie atau OVA dan episodes Unknow
anime.loc[(anime["type"]=="OVA") & (anime["episodes"]=="Unknown"),"episodes"] = "1" 
anime.loc[(anime["type"]=="Movie") & (anime["episodes"] == "Unknown"), "episodes"] = "1"

In [0]:
# Set nilai episodes pada data-data yang lain
fill_anime = {"Naruto Shippuuden":500, "One Piece":925,"Detective Conan":956, "Dragon Ball Super":131,
                "Crayon Shin chan":942, "Yu Gi Oh Arc V":148,"Shingeki no Kyojin Season 2":25,
                "Boku no Hero Academia 2nd Season":25,"Little Witch Academia TV":25}

# Looping set nilai
for k,v in fill_anime.items():    
    anime.loc[anime["name"]==k,"episodes"] = v

In [11]:
anime[(anime.episodes=="Unknown")]

,anime_id,name,genre,type,episodes,rating,members
615,1735,Naruto: Shippuuden,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,Unknown,7.94,533578
991,966,Crayon Shin-chan,"Comedy, Ecchi, Kids, School, Shounen, Slice of...",TV,Unknown,7.73,26267
1021,33157,Tanaka-kun wa Itsumo Kedaruge Specials,"Comedy, School, Slice of Life",Special,Unknown,7.72,5400
1272,21639,Yu☆Gi☆Oh! Arc-V,"Action, Fantasy, Game, Shounen",TV,Unknown,7.61,17571
1309,8687,Doraemon (2005),"Comedy, Kids, Sci-Fi, Shounen",TV,Unknown,7.59,2980
...,...,...,...,...,...,...,...
11105,34022,Whistle! (ONA),"School, Shounen, Sports",ONA,Unknown,NaN,381
11106,34467,Yami Shibai 4th Season,"Dementia, Horror, Supernatural",TV,Unknown,NaN,1838
11107,32615,Youjo Senki,"Magic, Military",TV,Unknown,NaN,6652
12252,34382,Citrus,"Drama, Romance, School, Yuri",NaN,Unknown,NaN,2459


In [0]:
# Set nilai episodes = mean yg "Unknow"
anime["episodes"] = anime["episodes"].map(lambda x:np.nan if x=="Unknown" else x)
anime["episodes"] = anime["episodes"].astype(float)
anime["episodes"].fillna(anime["episodes"].mean(),inplace = True)

# Set nilai rating = mean yg Nan
anime["rating"] = anime["rating"].astype(float)
anime["rating"].fillna(anime["rating"].mean(),inplace = True)

In [14]:
pd.get_dummies(anime[["type"]]).head()

,type_Movie,type_Music,type_ONA,type_OVA,type_Special,type_TV
0,1,0,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [15]:
anime["members"] = anime["members"].astype(float)
type(anime["members"][0])

numpy.float64

In [16]:
anime.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

## Membangun Features

In [0]:
# Merubah genre ke features
anime_features = pd.concat([anime["genre"].str.get_dummies(sep=","),
    pd.get_dummies(anime[["type"]]),
    anime[["rating"]],anime[["members"]],
    anime["episodes"]],axis=1)

In [18]:
anime_features.columns

Index([' Adventure', ' Cars', ' Comedy', ' Dementia', ' Demons', ' Drama',
       ' Ecchi', ' Fantasy', ' Game', ' Harem', ' Hentai', ' Historical',
       ' Horror', ' Josei', ' Kids', ' Magic', ' Martial Arts', ' Mecha',
       ' Military', ' Music', ' Mystery', ' Parody', ' Police',
       ' Psychological', ' Romance', ' Samurai', ' School', ' Sci-Fi',
       ' Seinen', ' Shoujo', ' Shoujo Ai', ' Shounen', ' Shounen Ai',
       ' Slice of Life', ' Space', ' Sports', ' Super Power', ' Supernatural',
       ' Thriller', ' Vampire', ' Yaoi', ' Yuri', 'Action', 'Adventure',
       'Cars', 'Comedy', 'Dementia', 'Demons', 'Drama', 'Ecchi', 'Fantasy',
       'Game', 'Harem', 'Hentai', 'Historical', 'Horror', 'Josei', 'Kids',
       'Magic', 'Martial Arts', 'Mecha', 'Military', 'Music', 'Mystery',
       'Parody', 'Police', 'Psychological', 'Romance', 'Samurai', 'School',
       'Sci-Fi', 'Seinen', 'Shoujo', 'Shounen', 'Slice of Life', 'Space',
       'Sports', 'Super Power', 'Supernatural'

In [0]:
# Scaling the data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
anime_features = scale.fit_transform(anime_features)

## Penerapan KNN

In [0]:
# KNN algorithm menemukan 11 data terdekat
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6).fit(anime_features)
distances, indices = nbrs.kneighbors(anime_features)

In [0]:
def get_index_from_name(name):
    return anime[anime["name"]==name].index.tolist()[0]

In [22]:
get_index_from_name('Pinkerton')

11426

In [0]:
all_anime_names = list(anime.name.values)

In [0]:
# Function mendapatkan id dari pencarian judul
def get_id_from_partial_name(partial):
    flag=0
    for name in all_anime_names:
        if partial.lower() in name.lower():
            print(name,", ID = ",all_anime_names.index(name))
            flag=1
            
    if(flag==0):
        print("No such anime found")

In [0]:
# Function memberikan 10 data rekomendasi berdasar id/judul
def get_recommendation_from_name(query):
        found_id = get_index_from_name(query)
        print(anime.iloc[found_id]["name"],"-",anime.iloc[found_id]["genre"]," >> ")
        for id in indices[found_id][1:]:
            print(anime.iloc[id]["name"],"-",anime.iloc[id]["genre"])
            
def get_recommendation_from_id(num):
    print(anime.iloc[num]["name"],"-",anime.iloc[num]["genre"]," >> ")
    for num in indices[num][1:]:
            print(anime.iloc[num]["name"],"-",anime.iloc[num]["genre"])

## Ujicoba dengan data

In [26]:
get_id_from_partial_name("Cowboy")

Cowboy Bebop , ID =  22
Cowboy Bebop: Tengoku no Tobira , ID =  152
Cowboy Bebop: Yose Atsume Blues , ID =  1465
Cowboy Bebop: Ein no Natsuyasumi , ID =  5815


In [27]:
get_recommendation_from_name("Cowboy Bebop")

Cowboy Bebop - Action, Adventure, Comedy, Drama, Sci-Fi, Space  >> 
Tengen Toppa Gurren Lagann - Action, Adventure, Comedy, Mecha, Sci-Fi
Seihou Bukyou Outlaw Star - Action, Adventure, Comedy, Sci-Fi, Space
Akame ga Kill! - Action, Adventure, Fantasy
D.Gray-man - Action, Adventure, Comedy, Shounen
Trigun - Action, Comedy, Sci-Fi


In [28]:
get_recommendation_from_id(22)

Cowboy Bebop - Action, Adventure, Comedy, Drama, Sci-Fi, Space  >> 
Tengen Toppa Gurren Lagann - Action, Adventure, Comedy, Mecha, Sci-Fi
Seihou Bukyou Outlaw Star - Action, Adventure, Comedy, Sci-Fi, Space
Akame ga Kill! - Action, Adventure, Fantasy
D.Gray-man - Action, Adventure, Comedy, Shounen
Trigun - Action, Comedy, Sci-Fi
